In [1]:
import utils_config
import os
import time
import datetime
import pandas as pd
import gspread as gs

from zenmoney import *
from sqlalchemy import create_engine


In [2]:
# Config File Path
config_path = os.getcwd() + "/config.json"
config = utils_config.load_config(config_path)


In [3]:
# MySQL Credentials
mysql_host = config.mysql.host
mysql_db = config.mysql.db
mysql_user = config.mysql.user
mysql_pwd = config.mysql.password


In [4]:
# Zenmoney Credentials – для прохождения Oauth
consumer_key = config.credentials.consumer_key
consumer_secret = config.credentials.consumer_secret
user_name = config.credentials.user_name
user_pass = config.credentials.user_pass


In [5]:
# MySQL Alchemy Engine – UTF8 SET!
mysql_engine = create_engine(
    f"mysql+mysqlconnector://{mysql_user}:{mysql_pwd}@{mysql_host}/{mysql_db}?charset=utf8")


In [ ]:
# В случае, если нужно делать какие-то запросы к самой базе
# select_statement = mysql_engine.execute("SELECT title, id FROM roma.accounts").fetchall() !!! FETCH ALL ()
# df_select = pd.DataFrame.from_dict(select_accounts)


In [6]:
# Zenmoney Authorization – но лучше вставить сам токен, который можно получить через Zerro (https://zerro.app/token)
# oauth = OAuth2(consumer_key, consumer_secret, user_name, user_pass)
zenmoney_api = Request(config.token)


In [7]:
# now = сегодня в формате Unix, который принимает сервер на вход
now = int(time.time())
diff_objects = zenmoney_api.diff(
    Diff(**{'serverTimestamp': 0, "currentClientTimestamp": now}))


In [8]:
# Разбираем на части объект account, его же сразу подгружаем внутрь sql и записываем с полем update_time, внутри будем хранить балансы и все счета и будет стата по дням!
df_account = pd.DataFrame.from_dict(diff_objects.account.to_dict())
df_account = df_account[['id',
                        'changed',
                         'instrument',
                         'type',
                         'inBalance',
                         'archive',
                         'private',
                         'savings',
                         'title',
                         'creditLimit',
                         'startBalance',
                         'balance']]
df_account['balance'] = df_account['balance'].astype('int')
df_account['startBalance'] = df_account['startBalance'].astype('int')

# Создаем серию данных, ограничиваем длиной получаемого df, сбрасываем индексы для стандартного join по индексу
update_s = pd.Series(int(time.time()), name='update_time').repeat(
    len(df_account)).reset_index(drop=True)
df_account = df_account.join(update_s)

# df_account['changed'] = df_account['changed'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d')) – не используется


In [ ]:
df_instruments = pd.DataFrame.from_dict(diff_objects.instrument.to_dict())
# Доллары, евро, рубли, другие валюты не будем собирать
df_instruments = df_instruments.loc[df_instruments['id'] < 4]
df_instruments = df_instruments[[
    'id', 'changed', 'shortTitle', 'symbol', 'rate']]

# df_instruments['changed'] = df_instruments['changed'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%d.%m.%Y')) - вдруг если нужно конвертнуть дату в самом датафрейме


In [ ]:
df_transactions = pd.DataFrame.from_dict(diff_objects.transaction.to_dict())
df_transactions = df_transactions[
    ['id',
     'changed',
     'created',
     'user',
     'deleted',
     'incomeInstrument',
     'incomeAccount',
     'income',
     'outcomeInstrument',
     'outcomeAccount',
     'outcome',
     'tag',
     'payee',
     'opIncome',
     'opIncomeInstrument',
     'opOutcome',
     'opOutcomeInstrument',
     'date',
     'originalPayee',
     'viewed',
     'hold',
     'qrCode',
     'source',
     'comment']
]

# Скидываем [] внутри колонки tag, чтобы нормально всё прогружать
df_transactions['tag'] = df_transactions['tag'].explode()


In [ ]:
# С тегами по-другому работаем, смотрим расходы по счетам, но на всякий тоже прогрузим

df_tags = pd.DataFrame.from_dict(diff_objects.tag.to_dict())
df_tags = df_tags[['id', 'changed', 'title', 'parent']]


In [ ]:
# Поехали грузить в SQL

df_account.to_sql('accounts', con=mysql_engine, if_exists='append')
df_instruments.to_sql('instruments', con=mysql_engine, if_exists='append')
df_transactions.to_sql('transactions', con=mysql_engine, if_exists='replace')
df_tags.to_sql('tags', con=mysql_engine, if_exists='replace')


In [1]:
# gs_sa = gs.service_account(filename=os.getcwd()+'/service_account.json')
# sheet = gs_sa.open_by_url(config.google_sheets.sheet_url)

# transactions_sheet = sheet.worksheet('transactions')
# # transactions_sheet.clear()
# transactions_sheet.update(
#     [df_transactions.columns.values.tolist()] + df_transactions.values.tolist())
